In [1]:
import pandas as pd
import numpy as np
import numpy.linalg as LA # operasi baris elementer
from scipy.sparse import linalg
from openpyxl import load_workbook # load data excel
from sklearn.feature_extraction.text import CountVectorizer # tf-idf
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer # tf-idf
from sklearn.metrics.pairwise import cosine_similarity # cosine similarity
from nltk.corpus import stopwords # preprocessing
from nltk.stem import PorterStemmer # preprocessing bahasa inggris
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # preprocessing
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #preprocessing
import string # ya buat string

In [3]:
ambildata = pd.read_excel("preprocessed-dataset.xlsx", sep=',')
ambildata

,Judul,Reviewer,Abstrak,preprocessed,preprocessed_judul,preprocessed_abstrak
0,impact of spam exposure on user engagement.,anirban dasgupta,In this paper we quantify the effect of unsoli...,impact spam exposure user engagement,impact spam exposure user engag,paper quantify effect unsolicited emails spam ...
1,a real time algorithm for detection of spectac...,anirban dasgupta,Eye detection plays an important role in many ...,real time algorithm detection spectacle lead e...,real time algorithm detection spectacles leadi...,eye detection plays important role many intell...
2,clustering cookies for identifying unique mobi...,anirban dasgupta,Embodiments are directed towards clustering co...,cluster cooky identify unique mobile device,clustering cookies identifying unique mobile d...,embodiments directed towards clustering cookie...
3,video & eog based investigation of pure saccad...,anirban dasgupta,Human Computer Interaction (HCI) is the method...,video eog base investigation pure saccade h...,video eog based investigation pure saccades hu...,human computer interaction hci methodology com...
4,spam or ham?: characterizing and detecting fra...,anirban dasgupta,"Web mail providers rely on users to\"" vote\"" t...",spam characterize detect fraudulent spam re...,spam ham characterizing detecting fraudulent s...,web mail providers rely users vote quickly col...
5,overcoming browser cookie churn with clustering,anirban dasgupta,Many large Internet websites are accessed by u...,overcome browser cookie churn cluster,overcoming browser cookie churn clust,many large internet websites accessed users an...
6,on sampling nodes in a network,anirban dasgupta,Random walk is an important tool in many graph...,sample node network,sampling nodes network,random walk important tool many graph mining a...
7,analysis of training parameters for classifier...,anirban dasgupta,This paper analyzes the performance of the Haa...,analysis train parameter classifier base featu...,analysis training parameters classifiers based...,paper analyzes performance haarlike feature ba...
8,an on-board vision based system for drowsiness...,anirban dasgupta,This paper proposes a system for on-board moni...,board vision base drowsiness detection automot...,onboard vision based system drowsiness detecti...,paper proposes system onboard monitoring loss ...
9,disambiguating authors in academic publication...,c. lee giles,Users of digital libraries usually want to kno...,disambiguate author academic publication rando...,disambiguating authors academic publications u...,users digital libraries usually want know exac...


In [3]:
def LoadDocuments(dPath=None,types=None, file = None): # types = ['pdf','doc','docx','txt','bz2']
    Files, Docs = [], []
    if types:
        for tipe in types:
            Files += crawlFiles(dPath,tipe)
    if file:
        Files = [file]
    if not types and not file: # get all files regardless of their extensions
        Files += crawlFiles(dPath)
    for f in Files:
        if f[-3:].lower()=='pdf':
            try:
                Docs.append(PDF(f).string)
            except:
                print('error reading{0}'.format(f))
        elif f[-3:].lower() in ['txt', 'dic','py', 'ipynb']:
            try:
                df=open(f,"r",encoding="utf-8", errors='replace')
                Docs.append(df.readlines());df.close()
            except:
                print('error reading{0}'.format(f))
        elif f[-3:].lower()=='bz2':
            try:
                Docs.append(readBz2(f))
            except:
                print('error reading{0}'.format(f))
        elif f[-4:].lower()=='docx':
            try:
                Docs.append(docx2txt.process(f))
            except:
                print('error reading{0}'.format(f))
        elif f[-3:].lower()=='csv':
            Docs.append(pd.read_csv(f))
        else:
            print('Unsupported format {0}'.format(f))
    if file:
        Docs = Docs[0]
    return Docs, Files
def LoadStopWords(lang='en'):
    L = lang.lower().strip()
    if L == 'en' or L == 'english' or L == 'inggris':
        from spacy.lang.en import English as lemmatizer
        #lemmatizer = spacy.lang.en.English
        lemmatizer = lemmatizer()
        #lemmatizer = spacy.load('en')
        stops =  set([t.strip() for t in LoadDocuments(file = 'data/stopwords_eng.txt')[0]])
    elif L == 'id' or L == 'indonesia' or L=='indonesian':
        from spacy.lang.id import Indonesian
        #lemmatizer = spacy.lang.id.Indonesian
        lemmatizer = Indonesian()
        stops = set([t.strip() for t in LoadDocuments(file = 'data/stopwords_id.txt')[0]])
    else:
        print('Warning, language not recognized. Empty StopWords Given')
        stops = set(); lemmatizer = None
    return stops, lemmatizer
def cleanText(T, fix={}, lemma=None, stops = set(), symbols_remove = True, min_charLen = 2, fixTag= True):
    # lang & stopS only 2 options : 'en' atau 'id'
    # symbols ASCII atau alnum
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    t = re.sub(pattern,' ',T) #remove urls if any
    pattern = re.compile(r'ftp[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    t = re.sub(pattern,' ',t) #remove urls if any
    t = unescape(t) # html entities fix
    if fixTag:
        t = fixTags(t) # fix abcDef
    t = t.lower().strip() # lowercase
    t = unidecode(t)
    t = ''.join(''.join(s)[:2] for _, s in itertools.groupby(t)) # remove repetition
    t = t.replace('\n', ' ').replace('\r', ' ')
    t = sent_tokenize(t) # sentence segmentation. String to list
    for i, K in enumerate(t):
        if symbols_remove:
            K = re.sub(r'[^.,_a-zA-Z0-9 \.]',' ',K)
        if lemma:
            listKata = lemma(K)
        else:
            listKata = TextBlob(K).words
        cleanList = []
        for token in listKata:
            if lemma:
                if str(token.text) in fix.keys():
                    token = fix[str(token.text)]
                try:
                    token = token.lemma_
                except:
                    token = lemma(token)[0].lemma_
            else:
                if str(token) in fix.keys():
                    token = fix[str(token)]
            if stops:
                if len(token)>=min_charLen and token not in stops:
                    cleanList.append(token)
            else:
                if len(token)>=min_charLen:
                    cleanList.append(token)
        t[i] = ' '.join(cleanList)
    return ' '.join(t) # Return kalimat lagi
def fixTags(t):
    getHashtags = re.compile(r"#(\w+)")
    pisahtags = re.compile(r'[A-Z][^A-Z]*')
    tagS = re.findall(getHashtags, t)
    for tag in tagS:
        if len(tag)>0:
            tg = tag[0].upper()+tag[1:]
            proper_words = []
            if adaAngka(tg):
                tag2 = re.split('(\d+)',tg)
                tag2 = [w for w in tag2 if len(w)>0]
                for w in tag2:
                    try:
                        _ = int(w) # error if w not a number
                        proper_words.append(w)
                    except:
                        w = w[0].upper()+w[1:]
                        proper_words = proper_words+re.findall(pisahtags, w)
            else:
                proper_words = re.findall(pisahtags, tg)
            proper_words = ' '.join(proper_words)
            t = t.replace('#'+tag, proper_words)
    return t

In [4]:
ambil= ambildata['Judul']

In [5]:
from tqdm import tqdm_notebook as tqdm
import re, networkx as nx, matplotlib.pyplot as plt, operator, numpy as np,community, os, docx2txt, csv#, spacy
from unidecode import unidecode
import json, pandas as pd, itertools, nltk, time
from nltk import sent_tokenize
from html import unescape


list_preprocessing_pada_judul_penelitian = []
print('Banyaknya produk = ', len(ambildata))

stops, lemmatizer = LoadStopWords(lang='id')
for i,d in tqdm(enumerate(ambil)):
    ambil[i] = cleanText(d, lemma=lemmatizer, stops = stops, symbols_remove = True, min_charLen = 2)
    list_preprocessing_pada_judul_penelitian.append(ambil[i])
print('Contoh deskripsi produk pertama = ', ambil)


Banyaknya produk =  4


A Jupyter Widget

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  



Contoh deskripsi produk pertama =  0    sistem learning berbasis model motivasi komunitas
1                  strategi kelola situs learning ajar
2           pilih sistem learning berbasis open source
3            kembang sistem learning multimedia gratis
Name: Judul, dtype: object


In [6]:
data = pd.read_csv("test.csv")
data

,Unnamed: 0,Judul,Lematizer
0,1,sistem learning berbasis model motivasi komunitas,sistem learning berbasis model motivasi komunitas
1,2,strategi kelola situs learning ajar,strategi kelola situs learning ajar
2,3,pilih sistem learning berbasis open source,pilih sistem learning berbasis open source
3,4,kembang sistem learning multimedia gratis,kembang sistem learning multimedia gratis


In [7]:
data = data["Lematizer"].values
data

array(['sistem learning berbasis model motivasi komunitas',
       'strategi kelola situs learning ajar',
       'pilih sistem learning berbasis open source',
       'kembang sistem learning multimedia gratis'], dtype=object)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(smooth_idf=False, norm=None)
tfidf = vectorizer.fit_transform(data)
words = vectorizer.get_feature_names()
similarity_matrix = cosine_similarity(tfidf)
for col in tfidf.nonzero()[1]:
    print (words[col], ' - ', tfidf[0, col])

sistem  -  1.2876820724517808
learning  -  1.0
berbasis  -  1.6931471805599454
model  -  2.386294361119891
motivasi  -  2.386294361119891
komunitas  -  2.386294361119891
learning  -  1.0
strategi  -  0.0
kelola  -  0.0
situs  -  0.0
ajar  -  0.0
sistem  -  1.2876820724517808
learning  -  1.0
berbasis  -  1.6931471805599454
pilih  -  0.0
open  -  0.0
source  -  0.0
sistem  -  1.2876820724517808
learning  -  1.0
kembang  -  0.0
multimedia  -  0.0
gratis  -  0.0


In [21]:
tfidf_matrix = vectorizer.fit_transform(data)
array = tfidf_matrix.todense()
df = pd.DataFrame(array)
df.head(4)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.000000,1.693147,0.000000,0.000000,0.000000,2.386294,1.0,2.386294,2.386294,0.000000,0.000000,0.000000,1.287682,0.000000,0.000000,0.000000
1,2.386294,0.000000,0.000000,2.386294,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.386294,0.000000,2.386294
2,0.000000,1.693147,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,2.386294,2.386294,1.287682,0.000000,2.386294,0.000000
3,0.000000,0.000000,2.386294,0.000000,2.386294,0.000000,1.0,0.000000,0.000000,2.386294,0.000000,0.000000,1.287682,0.000000,0.000000,0.000000


In [9]:
print(tfidf.shape)
print(tfidf.toarray())
print(tfidf.dtype)

(4, 16)
[[0.         1.69314718 0.         0.         0.         2.38629436
  1.         2.38629436 2.38629436 0.         0.         0.
  1.28768207 0.         0.         0.        ]
 [2.38629436 0.         0.         2.38629436 0.         0.
  1.         0.         0.         0.         0.         0.
  0.         2.38629436 0.         2.38629436]
 [0.         1.69314718 0.         0.         0.         0.
  1.         0.         0.         0.         2.38629436 2.38629436
  1.28768207 0.         2.38629436 0.        ]
 [0.         0.         2.38629436 0.         2.38629436 0.
  1.         0.         0.         2.38629436 0.         0.
  1.28768207 0.         0.         0.        ]]
float64


In [10]:
from scipy.sparse import linalg

U,S,V = linalg.svds(tfidf,k=2)
print("ukuran-ukuran matriks = ",U.shape, S.shape, V.shape)
print(type(U))

ukuran-ukuran matriks =  (4, 2) (2,) (2, 16)
<class 'numpy.ndarray'>


In [11]:
help(linalg)

Help on package scipy.sparse.linalg in scipy.sparse:

NAME
    scipy.sparse.linalg

DESCRIPTION
    Sparse linear algebra (:mod:`scipy.sparse.linalg`)
    
    .. currentmodule:: scipy.sparse.linalg
    
    Abstract linear operators
    -------------------------
    
    .. autosummary::
       :toctree: generated/
    
       LinearOperator -- abstract representation of a linear operator
       aslinearoperator -- convert an object to an abstract linear operator
    
    Matrix Operations
    -----------------
    
    .. autosummary::
       :toctree: generated/
    
       inv -- compute the sparse matrix inverse
       expm -- compute the sparse matrix exponential
       expm_multiply -- compute the product of a matrix exponential and a matrix
    
    Matrix norms
    ------------
    
    .. autosummary::
       :toctree: generated/
    
       norm -- Norm of a sparse matrix
       onenormest -- Estimate the 1-norm of a sparse matrix
    
    Solving linear problems
    -------

In [12]:
print(U)

[[ 0.19232655  0.63528677]
 [-0.9621556   0.26010311]
 [ 0.19232655  0.63528677]
 [ 0.01666137  0.35379063]]


In [13]:
print(S)

[4.8332705  5.47929567]


In [14]:
print(V)

[[-0.47503786  0.13474816  0.00822609 -0.47503786  0.00822609  0.09495594
  -0.11603761  0.09495594  0.09495594  0.00822609  0.09495594  0.09495594
   0.10691838 -0.47503786  0.09495594 -0.47503786]
 [ 0.1132778   0.39261762  0.15407976  0.1132778   0.15407976  0.27667447
   0.34392509  0.27667447  0.27667447  0.15407976  0.27667447  0.27667447
   0.38173969  0.1132778   0.27667447  0.1132778 ]]
